In [154]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
pd.set_option('display.max_columns',50) # DataFrame truncation 없이 보기
pd.set_option('display.max_rows',100)
import glob
import os
import re
from ast import literal_eval

In [155]:
paths = glob.glob('./data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops/*/reviews*')
paths

['./data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops\\1076\\reviews',
 './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops\\1076\\reviews%3fpage%3d0%26pageSize%3d20',
 './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops\\1076\\reviews%3fpage%3d1%26pageSize%3d20',
 './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops\\1076\\reviews%3fpage%3d10%26pageSize%3d20',
 './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops\\1076\\reviews%3fpage%3d11%26pageSize%3d20',
 './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops\\1076\\reviews%3fpage%3d12%26pageSize%3d20',
 './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops\\1076\\reviews%3fpage%3d13%26pageSize%3d20',
 './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops\\1076\\reviews%3fpage%3d14%26pageSize%3d20',
 './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops\\1076\\reviews%3fpage%3d2%26pageSize%3d20',
 './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops\\1076\

In [156]:
paths=[]
for root, _, files in os.walk('./data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops'):
    for file in files:
        if 'products' in root:
            continue
        if file[:7] == 'reviews':
            path = f'{root}/{file}'
            path = re.sub('\\\\', '/', path)
            paths.append(path)
paths

['./data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops/1076/reviews%3fpage%3d0%26pageSize%3d20',
 './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops/1076/reviews%3fpage%3d1%26pageSize%3d20',
 './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops/1076/reviews%3fpage%3d10%26pageSize%3d20',
 './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops/1076/reviews%3fpage%3d11%26pageSize%3d20',
 './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops/1076/reviews%3fpage%3d12%26pageSize%3d20',
 './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops/1076/reviews%3fpage%3d13%26pageSize%3d20',
 './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops/1076/reviews%3fpage%3d14%26pageSize%3d20',
 './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops/1076/reviews%3fpage%3d2%26pageSize%3d20',
 './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops/1076/reviews%3fpage%3d3%26pageSize%3d20',
 './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/sho

In [157]:
len(paths)

18

In [158]:
shop_ids=[]
for path in paths:
    shop_ids.append(path.split('/')[6])

shop_ids = set(shop_ids)
shop_ids

{'1076', '2085', '3843'}

In [159]:
paths_group_by_shopid = dict()
for shop_id in shop_ids:
    tmp = []
    for path in paths:
        if shop_id == path.split('/')[6]:
               tmp.append(path)
    paths_group_by_shopid[shop_id] = tmp

paths_group_by_shopid

{'3843': ['./data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops/3843/reviews'],
 '2085': ['./data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops/2085/reviews%3fpage%3d0%26pageSize%3d20',
  './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops/2085/reviews%3fpage%3d1%26pageSize%3d20'],
 '1076': ['./data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops/1076/reviews%3fpage%3d0%26pageSize%3d20',
  './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops/1076/reviews%3fpage%3d1%26pageSize%3d20',
  './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops/1076/reviews%3fpage%3d10%26pageSize%3d20',
  './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops/1076/reviews%3fpage%3d11%26pageSize%3d20',
  './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops/1076/reviews%3fpage%3d12%26pageSize%3d20',
  './data/review_scrap/hair-app.beauty.kakao.com(3rd)/v4/shops/1076/reviews%3fpage%3d13%26pageSize%3d20',
  './data/review_scrap/hair-app.beauty.kakao.com(3rd

In [160]:
def tf_capitalize(files: list):
    return [file.replace('true', 'True').replace('false', 'False') for file in files]

In [161]:
def remove_emoji(files: list):
    return [re.sub('\\\\[a-zA-Z0-9]+', ' ', file) for file in files]

In [162]:
def string2dict(files: list):
    return [literal_eval(file) for file in files]

In [163]:
files_group_by_shopid=dict()
for shop_id, paths in paths_group_by_shopid.items():
    tmp_files = []
    for path in paths:
        with open(path, 'rb') as f:
            file = f.read().decode('utf-8')
            tmp_files.append(file)
    
    # true, false => True, False로 변환
    tmp_files = tf_capitalize(tmp_files)

    # 이모지 제거
    tmp_files = remove_emoji(tmp_files)

    # string 객체를 dict 객체로 변환
    tmp_files = string2dict(tmp_files)
    
    files_group_by_shopid[shop_id] = tmp_files

In [164]:
print('크롤링된 업소 count :', len(files_group_by_shopid))

크롤링된 업소 count : 3


In [165]:
files_group_by_shopid.keys()

dict_keys(['3843', '2085', '1076'])

In [166]:
set([file['hasNext'] for file in files_group_by_shopid['2085']])

{False, True}

In [167]:
shopid_hasNext=[]
for shop_id, file in files_group_by_shopid.items():
    if False not in [file['hasNext'] for file in files_group_by_shopid[shop_id]]:
        shopid_hasNext.append(shop_id)
print('끝까지 못간 업소 count:', len(shopid_hasNext))
shopid_hasNext

끝까지 못간 업소 count: 0


[]

In [168]:
contents = []
for files in files_group_by_shopid.values():
    for file in files:
        contents += file['contents']

print('데이터 count :', len(contents))

데이터 count : 324


In [169]:
reviews = DataFrame(contents).sort_values(by='shopId')
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 324 entries, 161 to 0
Data columns (total 20 columns):
id                  324 non-null int64
comment             324 non-null object
reviewerName        324 non-null object
updateDate          324 non-null object
totalPoint          324 non-null float64
photoCount          324 non-null int64
reviewPhotos        324 non-null object
shopId              324 non-null int64
shopName            324 non-null object
stylerId            243 non-null float64
stylerName          243 non-null object
stylerLevel         243 non-null object
stylerActive        324 non-null bool
productName         324 non-null object
userChargedPrice    324 non-null int64
complianceStatus    324 non-null object
blindAt             324 non-null object
retreatment         324 non-null object
reply               324 non-null object
replyable           324 non-null bool
dtypes: bool(2), float64(2), int64(4), object(12)
memory usage: 48.7+ KB


In [170]:
reviews.tail(2)

,id,comment,reviewerName,updateDate,totalPoint,photoCount,reviewPhotos,shopId,shopName,stylerId,stylerName,stylerLevel,stylerActive,productName,userChargedPrice,complianceStatus,blindAt,retreatment,reply,replyable
25,113699,로레알 염색했습니다~^^ 친절히 계속 말걸어주시며 편안한 분위기로 머리 하고 왔어요...,김*림,2년 전,5.0,0,[],2085,더함헤어,8432.0,김기백,원장,True,로레알염색,0,NORMAL,,NONE,{},False
0,792873,다운펌 했는 데 완전 만족이에용~:D 앞으로 다닐 곳 정했습니당,이*민,9달 전,5.0,0,[],3843,청담동미용실,36159.0,정민주,실장,True,[컷기획전]커트+다운펌,0,NORMAL,,FRESH,{},False


In [171]:
reviews.to_csv('./kakao_hairshop_review_3rd.csv', sep=',', encoding='utf-8-sig')

In [172]:
len(reviews.shopId.unique())

3

In [173]:
kakao_review_1st = pd.read_csv('./kakao_hairshop_review_1st.csv', index_col=0)
kakao_review_1st.shape

C:\Users\ddoro\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(137010, 20)

In [174]:
kakao_review_2nd = pd.read_csv('./kakao_hairshop_review_2nd.csv', index_col=0)
kakao_review_2nd.shape

(55182, 20)

In [175]:
kakao_review_3rd = pd.read_csv('./kakao_hairshop_review_3rd.csv', index_col=0)
kakao_review_3rd.shape

(324, 20)

In [176]:
kakao_review_total = pd.concat([kakao_review_1st, kakao_review_2nd, kakao_review_3rd], ignore_index=True)
kakao_review_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192516 entries, 0 to 192515
Data columns (total 20 columns):
id                  192516 non-null int64
comment             187219 non-null object
reviewerName        192516 non-null object
updateDate          192516 non-null object
totalPoint          192516 non-null float64
photoCount          192516 non-null int64
reviewPhotos        192516 non-null object
shopId              192516 non-null int64
shopName            192516 non-null object
stylerId            137675 non-null float64
stylerName          137675 non-null object
stylerLevel         137647 non-null object
stylerActive        192516 non-null bool
productName         192516 non-null object
userChargedPrice    192516 non-null int64
complianceStatus    192516 non-null object
blindAt             5 non-null object
retreatment         192516 non-null object
reply               192515 non-null object
replyable           192516 non-null bool
dtypes: bool(2), float64(2), int64(4), o

In [177]:
kakao_review_total.drop_duplicates(inplace=True)
kakao_review_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158219 entries, 0 to 192515
Data columns (total 20 columns):
id                  158219 non-null int64
comment             153851 non-null object
reviewerName        158219 non-null object
updateDate          158219 non-null object
totalPoint          158219 non-null float64
photoCount          158219 non-null int64
reviewPhotos        158219 non-null object
shopId              158219 non-null int64
shopName            158219 non-null object
stylerId            114110 non-null float64
stylerName          114110 non-null object
stylerLevel         114083 non-null object
stylerActive        158219 non-null bool
productName         158219 non-null object
userChargedPrice    158219 non-null int64
complianceStatus    158219 non-null object
blindAt             5 non-null object
retreatment         158219 non-null object
reply               158218 non-null object
replyable           158219 non-null bool
dtypes: bool(2), float64(2), int64(4), o

In [178]:
#kakao_review_total.dropna(subset=['comment'], inplace=True)
#kakao_review_total.info()

In [179]:
kakao_review_total.head(2)

,id,comment,reviewerName,updateDate,totalPoint,photoCount,reviewPhotos,shopId,shopName,stylerId,stylerName,stylerLevel,stylerActive,productName,userChargedPrice,complianceStatus,blindAt,retreatment,reply,replyable
0,1089415,다온언니~ 늘 성실하게 잘해줘요~^^,이*주,3달 전,4.0,0,[],1,파란헤어 강남본점,6131.0,다온,수석 디자이너,True,뿌염+두피영양,0,NORMAL,NaN,RETREATMENT,"{'id': 690293, 'comment': '안녕하세요 고객님 디자이너 다온입니...",False
1,1162727,앞머리 너무 예쁘게 잘 잘라주셔서 감사해요☺️ 아미쌤 진짜 친절하시구 예쁘세용히히 ...,김*지,1달 전,5.0,0,[],1,파란헤어 강남본점,36947.0,아미,수석 디자이너,True,앞머리컷,0,NORMAL,NaN,RETREATMENT,"{'id': 744150, 'comment': '고객님 안녕하세요 아미디자이너...",False


In [180]:
kakao_reviews = kakao_review_total[['shopId', 'shopName', 'totalPoint', 'stylerName', 'stylerId','stylerLevel', 'productName', 'comment', 'reviewerName', 'updateDate']]
kakao_reviews

,shopId,shopName,totalPoint,stylerName,stylerId,stylerLevel,productName,comment,reviewerName,updateDate
0,1,파란헤어 강남본점,4.0,다온,6131.0,수석 디자이너,뿌염+두피영양,다온언니~ 늘 성실하게 잘해줘요~^^,이*주,3달 전
1,1,파란헤어 강남본점,5.0,아미,36947.0,수석 디자이너,앞머리컷,앞머리 너무 예쁘게 잘 잘라주셔서 감사해요☺️ 아미쌤 진짜 친절하시구 예쁘세용히히 ...,김*지,1달 전
2,1,파란헤어 강남본점,5.0,혜미,38986.0,디자이너,[3월픽]sbs생활의달인(복구염색or복구펌),정말 만족해요 색 진짜 예쁘게 잘 빠졌어요 ㅎㅎㅎ 긴 시간 고생하셨습니다~,김*연,1달 전
3,1,파란헤어 강남본점,5.0,수빈,4329.0,수석 디자이너,여성컷+뿌리염색+두피보호제,머리 해주시면서 설명도 잘해주시고 어떻게 관리해야 되는지 꼼꼼히 말해주셔서 너무 좋...,황*민,1달 전
4,1,파란헤어 강남본점,5.0,은정,29386.0,부원장,"디자인펌(남성,여성)+컷트",저보다 제 머리를 더 잘 아시는 분입니다. 펌을 해도 웨이브가 과하지 않으면서 딱...,노*호,1달 전
...,...,...,...,...,...,...,...,...,...,...
192511,2085,더함헤어,5.0,김기백,8432.0,원장,밀본염색,너무 편안히 예쁘게 머리바꾸고왔어요~~ 감사합니다 조만간클리닉받으러갈께요,임*린,2년 전
192512,2085,더함헤어,5.0,김기백,8432.0,원장,로레알뿌염,항상 친절하게 해주셔서 편하게 받을수있어서 좋아요~,홍*혜,2년 전
192513,2085,더함헤어,5.0,김기백,8432.0,원장,로레알뿌염,친절하시구 좋아요!!ㅎㅎ,홍*혜,2년 전
192514,2085,더함헤어,5.0,김기백,8432.0,원장,로레알염색,로레알 염색했습니다~^^ 친절히 계속 말걸어주시며 편안한 분위기로 머리 하고 왔어요...,김*림,2년 전


In [181]:
input_data = pd.read_csv('./data/kakao_hairshop_input_data_1st_2020-05-19_ansi.csv', encoding='ansi', index_col=0)
input_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 399 entries, 0 to 398
Data columns (total 16 columns):
id                          399 non-null int64
fullName                    399 non-null object
bestShop                    399 non-null bool
name                        399 non-null object
branchName                  372 non-null object
keyword                     390 non-null object
reviewTotalPoint            399 non-null float64
reviewCount                 399 non-null int64
favoriteCount               399 non-null int64
representativeColorPrice    365 non-null float64
representativeCutPrice      385 non-null float64
representativePermPrice     374 non-null float64
hotShop                     399 non-null bool
wgspointX                   399 non-null float64
wgspointY                   399 non-null float64
input_index                 399 non-null int64
dtypes: bool(2), float64(6), int64(4), object(4)
memory usage: 47.5+ KB


In [182]:
# map용 딕셔너리 생성
kakaoshopid_naverindex = Series(input_data.input_index.values, index=input_data.id).to_dict()
print(kakaoshopid_naverindex)

{3615: 0, 3658: 222, 2061: 11, 36: 27, 580: 1796, 7180: 2494, 1618: 2496, 3619: 164, 1853: 1911, 517: 2568, 2543: 168, 5703: 169, 4557: 254, 74: 737, 693: 2613, 687: 2617, 3684: 197, 7218: 2628, 7540: 170, 758: 171, 1303: 2670, 2085: 2671, 4392: 2675, 2242: 2680, 6743: 1730, 2067: 113, 3667: 646, 8355: 1518, 7422: 329, 158: 369, 185: 333, 7395: 1618, 3623: 305, 624: 16, 7423: 2720, 3651: 175, 3620: 28, 3689: 37, 4790: 391, 3648: 221, 3344: 179, 3630: 185, 8595: 189, 6257: 190, 7689: 191, 3652: 193, 3621: 194, 3677: 29, 1768: 196, 3681: 199, 2310: 201, 771: 204, 2499: 205, 1076: 208, 4508: 211, 8559: 213, 4949: 216, 3680: 217, 7811: 218, 260: 219, 108: 220, 240: 31, 886: 225, 5800: 226, 3004: 227, 3641: 233, 3804: 236, 4409: 242, 927: 32, 3653: 243, 2293: 244, 3691: 248, 111: 249, 2364: 253, 2710: 259, 4694: 261, 6311: 264, 2443: 266, 3628: 267, 4619: 270, 175: 273, 890: 276, 79: 279, 3696: 281, 6977: 35, 3179: 289, 2909: 290, 4813: 291, 5825: 292, 2767: 297, 6141: 300, 8584: 308, 526: 

In [183]:
kakao_reviews['naver_index'] = kakao_reviews.shopId.map(kakaoshopid_naverindex)
kakao_reviews.head(2)

C:\Users\ddoro\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,shopId,shopName,totalPoint,stylerName,stylerId,stylerLevel,productName,comment,reviewerName,updateDate,naver_index
0,1,파란헤어 강남본점,4.0,다온,6131.0,수석 디자이너,뿌염+두피영양,다온언니~ 늘 성실하게 잘해줘요~^^,이*주,3달 전,39.0
1,1,파란헤어 강남본점,5.0,아미,36947.0,수석 디자이너,앞머리컷,앞머리 너무 예쁘게 잘 잘라주셔서 감사해요☺️ 아미쌤 진짜 친절하시구 예쁘세용히히 ...,김*지,1달 전,39.0


In [184]:
kakao_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158219 entries, 0 to 192515
Data columns (total 11 columns):
shopId          158219 non-null int64
shopName        158219 non-null object
totalPoint      158219 non-null float64
stylerName      114110 non-null object
stylerId        114110 non-null float64
stylerLevel     114083 non-null object
productName     158219 non-null object
comment         153851 non-null object
reviewerName    158219 non-null object
updateDate      158219 non-null object
naver_index     157466 non-null float64
dtypes: float64(3), int64(1), object(7)
memory usage: 14.5+ MB


In [185]:
kakao_reviews.dropna(subset=['naver_index'], inplace=True)

C:\Users\ddoro\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [186]:
kakao_reviews.naver_index = kakao_reviews.naver_index.astype('int64')

C:\Users\ddoro\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [187]:
kakao_reviews

,shopId,shopName,totalPoint,stylerName,stylerId,stylerLevel,productName,comment,reviewerName,updateDate,naver_index
0,1,파란헤어 강남본점,4.0,다온,6131.0,수석 디자이너,뿌염+두피영양,다온언니~ 늘 성실하게 잘해줘요~^^,이*주,3달 전,39
1,1,파란헤어 강남본점,5.0,아미,36947.0,수석 디자이너,앞머리컷,앞머리 너무 예쁘게 잘 잘라주셔서 감사해요☺️ 아미쌤 진짜 친절하시구 예쁘세용히히 ...,김*지,1달 전,39
2,1,파란헤어 강남본점,5.0,혜미,38986.0,디자이너,[3월픽]sbs생활의달인(복구염색or복구펌),정말 만족해요 색 진짜 예쁘게 잘 빠졌어요 ㅎㅎㅎ 긴 시간 고생하셨습니다~,김*연,1달 전,39
3,1,파란헤어 강남본점,5.0,수빈,4329.0,수석 디자이너,여성컷+뿌리염색+두피보호제,머리 해주시면서 설명도 잘해주시고 어떻게 관리해야 되는지 꼼꼼히 말해주셔서 너무 좋...,황*민,1달 전,39
4,1,파란헤어 강남본점,5.0,은정,29386.0,부원장,"디자인펌(남성,여성)+컷트",저보다 제 머리를 더 잘 아시는 분입니다. 펌을 해도 웨이브가 과하지 않으면서 딱...,노*호,1달 전,39
...,...,...,...,...,...,...,...,...,...,...,...
192511,2085,더함헤어,5.0,김기백,8432.0,원장,밀본염색,너무 편안히 예쁘게 머리바꾸고왔어요~~ 감사합니다 조만간클리닉받으러갈께요,임*린,2년 전,2671
192512,2085,더함헤어,5.0,김기백,8432.0,원장,로레알뿌염,항상 친절하게 해주셔서 편하게 받을수있어서 좋아요~,홍*혜,2년 전,2671
192513,2085,더함헤어,5.0,김기백,8432.0,원장,로레알뿌염,친절하시구 좋아요!!ㅎㅎ,홍*혜,2년 전,2671
192514,2085,더함헤어,5.0,김기백,8432.0,원장,로레알염색,로레알 염색했습니다~^^ 친절히 계속 말걸어주시며 편안한 분위기로 머리 하고 왔어요...,김*림,2년 전,2671


In [188]:
len(kakao_reviews.shopId.unique())

399

In [189]:
naver_path = 'D:/bigdata10/99.team_project/데이터셋/요청 함수/naverplace_blog_review/naver_place_count_total_2020-05-01_final.csv'
naver_data = pd.read_csv(naver_path, index_col=0)

In [190]:
naverindex_naverstoreid = naver_data.store_id_only_num.to_dict()

In [191]:
kakao_reviews['store_id_only_num'] = kakao_reviews.naver_index.map(naverindex_naverstoreid)

C:\Users\ddoro\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [192]:
kakao_reviews

,shopId,shopName,totalPoint,stylerName,stylerId,stylerLevel,productName,comment,reviewerName,updateDate,naver_index,store_id_only_num
0,1,파란헤어 강남본점,4.0,다온,6131.0,수석 디자이너,뿌염+두피영양,다온언니~ 늘 성실하게 잘해줘요~^^,이*주,3달 전,39,31197568
1,1,파란헤어 강남본점,5.0,아미,36947.0,수석 디자이너,앞머리컷,앞머리 너무 예쁘게 잘 잘라주셔서 감사해요☺️ 아미쌤 진짜 친절하시구 예쁘세용히히 ...,김*지,1달 전,39,31197568
2,1,파란헤어 강남본점,5.0,혜미,38986.0,디자이너,[3월픽]sbs생활의달인(복구염색or복구펌),정말 만족해요 색 진짜 예쁘게 잘 빠졌어요 ㅎㅎㅎ 긴 시간 고생하셨습니다~,김*연,1달 전,39,31197568
3,1,파란헤어 강남본점,5.0,수빈,4329.0,수석 디자이너,여성컷+뿌리염색+두피보호제,머리 해주시면서 설명도 잘해주시고 어떻게 관리해야 되는지 꼼꼼히 말해주셔서 너무 좋...,황*민,1달 전,39,31197568
4,1,파란헤어 강남본점,5.0,은정,29386.0,부원장,"디자인펌(남성,여성)+컷트",저보다 제 머리를 더 잘 아시는 분입니다. 펌을 해도 웨이브가 과하지 않으면서 딱...,노*호,1달 전,39,31197568
...,...,...,...,...,...,...,...,...,...,...,...,...
192511,2085,더함헤어,5.0,김기백,8432.0,원장,밀본염색,너무 편안히 예쁘게 머리바꾸고왔어요~~ 감사합니다 조만간클리닉받으러갈께요,임*린,2년 전,2671,20888578
192512,2085,더함헤어,5.0,김기백,8432.0,원장,로레알뿌염,항상 친절하게 해주셔서 편하게 받을수있어서 좋아요~,홍*혜,2년 전,2671,20888578
192513,2085,더함헤어,5.0,김기백,8432.0,원장,로레알뿌염,친절하시구 좋아요!!ㅎㅎ,홍*혜,2년 전,2671,20888578
192514,2085,더함헤어,5.0,김기백,8432.0,원장,로레알염색,로레알 염색했습니다~^^ 친절히 계속 말걸어주시며 편안한 분위기로 머리 하고 왔어요...,김*림,2년 전,2671,20888578


In [193]:
kakao_reviews.describe()

,shopId,totalPoint,stylerId,naver_index,store_id_only_num
count,157466.000000,157466.000000,113551.000000,157466.000000,1.574660e+05
mean,2440.338752,4.881160,21088.729382,571.653525,5.579449e+08
std,1775.906842,0.436199,11957.975415,582.140943,6.923640e+08
min,1.000000,1.000000,132.000000,0.000000,1.157438e+07
25%,755.000000,5.000000,12457.000000,137.000000,3.393761e+07
50%,2395.000000,5.000000,20503.000000,348.000000,3.831416e+07
75%,3653.000000,5.000000,30287.000000,871.000000,1.183343e+09
max,8666.000000,5.000000,48334.000000,2720.000000,1.991884e+09


In [194]:
kakao_reviews.drop('naver_index', axis=1, inplace=True)
kakao_reviews

C:\Users\ddoro\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,shopId,shopName,totalPoint,stylerName,stylerId,stylerLevel,productName,comment,reviewerName,updateDate,store_id_only_num
0,1,파란헤어 강남본점,4.0,다온,6131.0,수석 디자이너,뿌염+두피영양,다온언니~ 늘 성실하게 잘해줘요~^^,이*주,3달 전,31197568
1,1,파란헤어 강남본점,5.0,아미,36947.0,수석 디자이너,앞머리컷,앞머리 너무 예쁘게 잘 잘라주셔서 감사해요☺️ 아미쌤 진짜 친절하시구 예쁘세용히히 ...,김*지,1달 전,31197568
2,1,파란헤어 강남본점,5.0,혜미,38986.0,디자이너,[3월픽]sbs생활의달인(복구염색or복구펌),정말 만족해요 색 진짜 예쁘게 잘 빠졌어요 ㅎㅎㅎ 긴 시간 고생하셨습니다~,김*연,1달 전,31197568
3,1,파란헤어 강남본점,5.0,수빈,4329.0,수석 디자이너,여성컷+뿌리염색+두피보호제,머리 해주시면서 설명도 잘해주시고 어떻게 관리해야 되는지 꼼꼼히 말해주셔서 너무 좋...,황*민,1달 전,31197568
4,1,파란헤어 강남본점,5.0,은정,29386.0,부원장,"디자인펌(남성,여성)+컷트",저보다 제 머리를 더 잘 아시는 분입니다. 펌을 해도 웨이브가 과하지 않으면서 딱...,노*호,1달 전,31197568
...,...,...,...,...,...,...,...,...,...,...,...
192511,2085,더함헤어,5.0,김기백,8432.0,원장,밀본염색,너무 편안히 예쁘게 머리바꾸고왔어요~~ 감사합니다 조만간클리닉받으러갈께요,임*린,2년 전,20888578
192512,2085,더함헤어,5.0,김기백,8432.0,원장,로레알뿌염,항상 친절하게 해주셔서 편하게 받을수있어서 좋아요~,홍*혜,2년 전,20888578
192513,2085,더함헤어,5.0,김기백,8432.0,원장,로레알뿌염,친절하시구 좋아요!!ㅎㅎ,홍*혜,2년 전,20888578
192514,2085,더함헤어,5.0,김기백,8432.0,원장,로레알염색,로레알 염색했습니다~^^ 친절히 계속 말걸어주시며 편안한 분위기로 머리 하고 왔어요...,김*림,2년 전,20888578


In [197]:
kakao_reviews.index = range(len(kakao_reviews))
kakao_reviews

,shopId,shopName,totalPoint,stylerName,stylerId,stylerLevel,productName,comment,reviewerName,updateDate,store_id_only_num
0,1,파란헤어 강남본점,4.0,다온,6131.0,수석 디자이너,뿌염+두피영양,다온언니~ 늘 성실하게 잘해줘요~^^,이*주,3달 전,31197568
1,1,파란헤어 강남본점,5.0,아미,36947.0,수석 디자이너,앞머리컷,앞머리 너무 예쁘게 잘 잘라주셔서 감사해요☺️ 아미쌤 진짜 친절하시구 예쁘세용히히 ...,김*지,1달 전,31197568
2,1,파란헤어 강남본점,5.0,혜미,38986.0,디자이너,[3월픽]sbs생활의달인(복구염색or복구펌),정말 만족해요 색 진짜 예쁘게 잘 빠졌어요 ㅎㅎㅎ 긴 시간 고생하셨습니다~,김*연,1달 전,31197568
3,1,파란헤어 강남본점,5.0,수빈,4329.0,수석 디자이너,여성컷+뿌리염색+두피보호제,머리 해주시면서 설명도 잘해주시고 어떻게 관리해야 되는지 꼼꼼히 말해주셔서 너무 좋...,황*민,1달 전,31197568
4,1,파란헤어 강남본점,5.0,은정,29386.0,부원장,"디자인펌(남성,여성)+컷트",저보다 제 머리를 더 잘 아시는 분입니다. 펌을 해도 웨이브가 과하지 않으면서 딱...,노*호,1달 전,31197568
...,...,...,...,...,...,...,...,...,...,...,...
157461,2085,더함헤어,5.0,김기백,8432.0,원장,밀본염색,너무 편안히 예쁘게 머리바꾸고왔어요~~ 감사합니다 조만간클리닉받으러갈께요,임*린,2년 전,20888578
157462,2085,더함헤어,5.0,김기백,8432.0,원장,로레알뿌염,항상 친절하게 해주셔서 편하게 받을수있어서 좋아요~,홍*혜,2년 전,20888578
157463,2085,더함헤어,5.0,김기백,8432.0,원장,로레알뿌염,친절하시구 좋아요!!ㅎㅎ,홍*혜,2년 전,20888578
157464,2085,더함헤어,5.0,김기백,8432.0,원장,로레알염색,로레알 염색했습니다~^^ 친절히 계속 말걸어주시며 편안한 분위기로 머리 하고 왔어요...,김*림,2년 전,20888578


In [199]:
kakao_reviews.to_csv('./kakaohairshop_mobileapp_review.csv', sep=',', encoding='utf-8-sig')
     